<a href="https://colab.research.google.com/github/Kenton-Tang-HK/Trading-Strategies-with-Random-Forest-and-LightGBM/blob/main/LightGBMbasedStrategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -nc https://lazyprogrammer.me/course_files/sp500_closefull.csv

--2024-02-09 13:24:02--  https://lazyprogrammer.me/course_files/sp500_closefull.csv
Resolving lazyprogrammer.me (lazyprogrammer.me)... 172.67.213.166, 104.21.23.210, 2606:4700:3031::6815:17d2, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|172.67.213.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18464866 (18M) [text/csv]
Saving to: ‘sp500_closefull.csv’

sp500_closefull.csv 100%[===================>]  17.61M  23.4MB/s    in 0.8s    

2024-02-09 13:24:03 (23.4 MB/s) - ‘sp500_closefull.csv’ saved [18464866/18464866]



Import Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Data Wrangling

In [3]:
df = pd.read_csv('sp500_closefull.csv', index_col=0, parse_dates=True)
df.head()

,CSCO,UAL,TROW,ISRG,PRGO,TPR,DVN,MRO,BA,VRTX,...,M,CRM,PGR,WAT,BWA,LRCX,NWL,UAA,BLK,PPL
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,24.690001,12.80,54.400002,102.923332,40.349998,36.310001,76.570000,19.153616,56.180000,44.240002,...,17.059999,18.705000,18.030001,61.630001,16.889999,39.880001,15.20,3.51000,238.580002,30.242558
2010-01-05,24.580000,13.91,55.009998,102.459999,38.790001,36.750000,76.650002,19.171511,58.020000,42.779999,...,16.860001,18.625000,17.969999,60.790001,17.695000,39.610001,15.11,3.61500,239.610001,29.851370
2010-01-06,24.420000,13.27,54.150002,103.946663,38.299999,37.470001,76.419998,19.595024,59.779999,42.029999,...,17.100000,18.592501,17.790001,60.900002,18.344999,39.430000,15.38,3.69500,234.669998,29.916569
2010-01-07,24.530001,13.55,54.110001,103.556664,37.990002,37.490002,75.970001,19.475725,62.200001,41.500000,...,17.490000,18.510000,17.549999,61.160000,18.594999,39.360001,15.82,3.65125,237.250000,29.627834
2010-01-08,24.660000,13.33,53.900002,102.986664,37.779999,37.270000,76.120003,19.505550,61.599998,40.669998,...,16.920000,18.537500,17.709999,61.209999,18.254999,40.349998,15.77,3.64375,238.919998,29.534695


In [4]:
df.dropna(axis=0, how='all', inplace=True)
df.shape

(2263, 480)

In [5]:
df.dropna(axis=1, how='any', inplace=True)
df.shape

(2263, 429)

In [6]:
df.isna().sum().sum()

0

In [7]:
# Calculate log return
df_returns = pd.DataFrame()
for name in df.columns:
  df_returns[name] = np.log(df[name]).diff()

<ipython-input-7-cb022972688f>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_returns[name] = np.log(df[name]).diff()
<ipython-input-7-cb022972688f>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_returns[name] = np.log(df[name]).diff()
<ipython-input-7-cb022972688f>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `ne

In [8]:
df_returns.head()

,CSCO,UAL,TROW,ISRG,PRGO,TPR,DVN,MRO,BA,VRTX,...,M,CRM,PGR,WAT,BWA,LRCX,NWL,UAA,BLK,PPL
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-05,-0.004465,0.083163,0.011151,-0.004512,-0.039429,0.012045,0.001044,0.000934,0.032227,-0.033559,...,-0.011793,-0.004286,-0.003333,-0.013723,0.046560,-0.006793,-0.005939,0.029476,0.004308,-0.013019
2010-01-06,-0.006531,-0.047102,-0.015757,0.014405,-0.012713,0.019402,-0.003005,0.021850,0.029883,-0.017687,...,0.014134,-0.001746,-0.010067,0.001808,0.036075,-0.004555,0.017711,0.021889,-0.020832,0.002182
2010-01-07,0.004494,0.020881,-0.000739,-0.003759,-0.008127,0.000534,-0.005906,-0.006107,0.039684,-0.012690,...,0.022551,-0.004447,-0.013583,0.004260,0.013536,-0.001777,0.028207,-0.011911,0.010934,-0.009698
2010-01-08,0.005286,-0.016369,-0.003889,-0.005519,-0.005543,-0.005886,0.001973,0.001530,-0.009693,-0.020203,...,-0.033133,0.001485,0.009075,0.000817,-0.018454,0.024841,-0.003166,-0.002056,0.007014,-0.003149


In [9]:
df_returns['SPY'] = df_returns['SPY'].shift(-1)
df_returns['SPY'].tail()

Date
2018-12-21   -0.026778
2018-12-24    0.049290
2018-12-26    0.007648
2018-12-27   -0.001291
2018-12-28         NaN
Name: SPY, dtype: float64

In [10]:
# train-test split
Ntest = 1000
train = df_returns.iloc[1:-Ntest]
test = df_returns.iloc[-Ntest:-1]

In [11]:
df_returns.columns

Index(['CSCO', 'UAL', 'TROW', 'ISRG', 'PRGO', 'TPR', 'DVN', 'MRO', 'BA',
       'VRTX',
       ...
       'M', 'CRM', 'PGR', 'WAT', 'BWA', 'LRCX', 'NWL', 'UAA', 'BLK', 'PPL'],
      dtype='object', length=429)

In [12]:
# Features
x_cols = ['AAPL', 'MSFT', 'AMZN', 'JNJ', 'V', 'PG', 'JPM']
x_cols

['AAPL', 'MSFT', 'AMZN', 'JNJ', 'V', 'PG', 'JPM']

In [13]:
Xtrain = train[x_cols]
Ytrain = train['SPY']
Xtest = test[x_cols]
Ytest = test['SPY']

Xtrain.head()

,AAPL,MSFT,AMZN,JNJ,V,PG,JPM
Date,,,,,,,
2010-01-05,0.001727,0.000323,0.005883,-0.011663,-0.011525,0.000327,0.019185
2010-01-06,-0.016034,-0.006156,-0.018282,0.008101,-0.013519,-0.004755,0.005479
2010-01-07,-0.001850,-0.010454,-0.017160,-0.007163,0.009264,-0.005438,0.019615
2010-01-08,0.006626,0.006873,0.026717,0.003432,0.002762,-0.001323,-0.002459
2010-01-11,-0.008861,-0.012802,-0.024335,0.000156,-0.002878,-0.003979,-0.003363


In [14]:
Ytrain.head()

Date
2010-01-05    0.000704
2010-01-06    0.004212
2010-01-07    0.003322
2010-01-08    0.001396
2010-01-11   -0.009370
Name: SPY, dtype: float64

Linear Regression


In [15]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(Xtrain, Ytrain)
model.score(Xtrain, Ytrain), model.score(Xtest, Ytest)

(0.008271754178782675, -0.011369618185063324)

In [16]:
# Direction
Ptrain = model.predict(Xtrain)
Ptest = model.predict(Xtest)
np.mean(np.sign(Ptrain) == np.sign(Ytrain)), np.mean(np.sign(Ptest) == np.sign(Ytest))

(0.528526148969889, 0.5285285285285285)

In [18]:
train_idx = df.index <= train.index[-1]
test_idx = df.index > train.index[-1]

train_idx[0] = False
test_idx[-1] = False

In [23]:
df_returns['Position'] = 0 # create new column
df_returns.loc[train_idx,'Position'] = (Ptrain > 0)
df_returns.loc[test_idx,'Position'] = (Ptest > 0)
df_returns['AlgoReturn'] = df_returns['Position'] * df_returns['SPY']
# Total algo log return train
df_returns.iloc[1:-Ntest]['AlgoReturn'].sum()

0.5477174926058392

In [24]:
# Total algo log return test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].sum()

0.3037759916491991

In [25]:
# Total return buy-and-hold train
Ytrain.sum()

0.5863972019814705

In [26]:
# Total return buy-and-hold test
Ytest.sum()

0.19307543946998518

Logistic Regression

In [27]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(C=10)
Ctrain = (Ytrain > 0)
Ctest = (Ytest > 0)
model.fit(Xtrain, Ctrain)
model.score(Xtrain, Ctrain), model.score(Xtest, Ctest)

(0.5586370839936609, 0.5285285285285285)

In [28]:
Ptrain = model.predict(Xtrain)
Ptest = model.predict(Xtest)
set(Ptrain), set(Ptest)

({False, True}, {False, True})

In [29]:
df_returns.loc[train_idx,'Position'] = Ptrain
df_returns.loc[test_idx,'Position'] = Ptest
df_returns['AlgoReturn'] = df_returns['Position'] * df_returns['SPY']
# Total algo log return train
df_returns.iloc[1:-Ntest]['AlgoReturn'].sum()

0.5905718759742236

In [30]:
# Total algo log return test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].sum()

0.19680747922142316

In [31]:
# Total return buy-and-hold
Ytrain.sum(), Ytest.sum()

(0.5863972019814705, 0.19307543946998518)

Random Forest

In [32]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=2)
model.fit(Xtrain, Ctrain)
model.score(Xtrain, Ctrain), model.score(Xtest, Ctest)

(1.0, 0.5255255255255256)

In [33]:
Ptrain = model.predict(Xtrain)
Ptest = model.predict(Xtest)
df_returns.loc[train_idx,'Position'] = Ptrain
df_returns.loc[test_idx,'Position'] = Ptest
df_returns['AlgoReturn'] = df_returns['Position'] * df_returns['SPY']
# Total algo log return train
df_returns.iloc[1:-Ntest]['AlgoReturn'].sum()

4.706503557502187

In [34]:
# Total algo log return test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].sum()

0.3839643540537496

In [35]:
# Total return buy-and-hold
Ytrain.sum(), Ytest.sum()

(0.5863972019814705, 0.19307543946998518)

LightBGM

In [36]:
!pip install lightgbm

In [37]:
import lightgbm as lgb

dtrain = lgb.Dataset(Xtrain, label=Ctrain)
dtest = lgb.Dataset(Xtest, label=Ctest, reference=dtrain)

params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

# Train the model
num_round = 100  # Number of boosting rounds
bst = lgb.train(params, dtrain, num_round, valid_sets=[dtest])

In [38]:
# Generating predictions
Ptrain = bst.predict(Xtrain, num_iteration=bst.best_iteration)
Ptest = bst.predict(Xtest, num_iteration=bst.best_iteration)

# Assuming binary classification, converting probabilities to class labels (0 or 1)
Ptrain_labels = (Ptrain > 0.5).astype(int)
Ptest_labels = (Ptest > 0.5).astype(int)

# Update positions based on predictions
df_returns.loc[train_idx, 'Position'] = Ptrain_labels
df_returns.loc[test_idx, 'Position'] = Ptest_labels

# Calculate algo returns
df_returns['AlgoReturn'] = df_returns['Position'] * df_returns['SPY']

# Calculate total algo log return for train and test sets
train_algo_log_return = df_returns.iloc[1:-Ntest]['AlgoReturn'].sum()
test_algo_log_return = df_returns.iloc[-Ntest:-1]['AlgoReturn'].sum()

# Calculate total return buy-and-hold for train and test sets
train_buy_and_hold_return = Ytrain.sum()
test_buy_and_hold_return = Ytest.sum()

print("Train Algo Log Return:", train_algo_log_return)
print("Test Algo Log Return:", test_algo_log_return)
print("Train Buy and Hold Return:", train_buy_and_hold_return)
print("Test Buy and Hold Return:", test_buy_and_hold_return)

Train Algo Log Return: 4.1085475572329
Test Algo Log Return: 0.40340388488900025
Train Buy and Hold Return: 0.5863972019814705
Test Buy and Hold Return: 0.19307543946998518
